# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [0]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

#Custom Imports
import heapq
import pandas as pd
from IPython.display import display, HTML
from html import unescape
from scipy.stats import percentileofscore

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [0]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = int(len(newsgroups_test.target) / 2)
test_data, test_labels = newsgroups_test.data[num_test:], newsgroups_test.target[num_test:]
dev_data, dev_labels = newsgroups_test.data[:num_test], newsgroups_test.target[:num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

### Part 1:

For each of the first 5 training examples, print the text of the message along with the label.

In [0]:
def P1(num_examples=5):
  for i in range(num_examples):
    print("---------------------------------------------------------------------------------")
    print("Message " + str(i + 1) + ", Label: " + newsgroups_train.target_names[train_labels[i]])
    print("---------------------------------------------------------------------------------")
    print(train_data[i] + "\n\n")

P1()

### Part 2:

Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [0]:
def P2():
  vectorizer = CountVectorizer()
  vectors = vectorizer.fit_transform(train_data)
  
  # nnz returns number of non-zero entries, shape[0] is number of examples
  print("Avg Unique Words Per Document: " + str(float(vectors.nnz) / vectors.shape[0]))
  # full matrix is shape[0](# docs)*shape[1](total vocab size)
  print("Fraction of Non-Zero Matrix Entries: " + str(float(vectors.nnz) / (vectors.shape[0]*vectors.shape[1])))
  
  # get_feature_names returns all features in alphabetical order
  features = vectorizer.get_feature_names()
  print("0th Feature: '" + features[0] + "'")
  print("Last Feature: '" + features[-1] + "'")
  
  vocab = ["atheism", "graphics", "space", "religion"]
  vectorizer_with_vocab = CountVectorizer(vocabulary = vocab)
  vectors_with_vocab = vectorizer_with_vocab.fit_transform(train_data)
  
  print("Avg Unique Words from Vocab of " + str(vocab) + ": " + str(float(vectors_with_vocab.nnz) / vectors_with_vocab.shape[0]))
  
  bi_tri_vctzr = CountVectorizer(analyzer = 'char', ngram_range = (2,3))
  bi_tri_vctrs = bi_tri_vctzr.fit_transform(train_data)
  
  print("Number of Bi- and Tri-gram Character Features: " + str(bi_tri_vctrs.shape[1]))
  
  vctzr_df10 = CountVectorizer(min_df = 10)
  vctrs_df10 = vctzr_df10.fit_transform(train_data)
  
  print("Number of Words in At Least 10 Documents: " + str(vctrs_df10.shape[1]))
  
  dev_vectors = vectorizer.fit_transform(dev_data)
  dev_features = vectorizer.get_feature_names()
  
  dev_only_features = set(dev_features) - set(features)
  dev_only_fraction = float(len(dev_only_features)) / len(dev_features)
  print("Fraction of Words in Dev Data Missing From Training Vocab: " + str(dev_only_fraction))
  

P2()

### Part 3:

Use the default CountVectorizer options and report the f1 score (use metrics.f1_score with average="weighted") for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

* Why doesn't nearest neighbors work well for this problem?
* Any ideas why logistic regression doesn't work as well as Naive Bayes?
* Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [0]:
def P3():
  train_vectorizer = CountVectorizer()
  train_vectors = train_vectorizer.fit_transform(train_data)
  train_features = train_vectorizer.get_feature_names()
  
  dev_vectorizer = CountVectorizer(vocabulary=train_features)
  dev_vectors = dev_vectorizer.fit_transform(dev_data)
  
  search_knn = KNeighborsClassifier()
  params_knn = { 'n_neighbors': [n for n in range(1, 100, 1)] }
  gscv_knn = GridSearchCV(search_knn, params_knn, cv=5, scoring='f1_weighted')
  gscv_knn.fit(train_vectors, train_labels)
  print("Best k-Value for kNN via GridSearchCV: " + str(gscv_knn.best_params_))
  dev_pred_labels_knn = gscv_knn.predict(dev_vectors)
  print("f1-score for kNN: " + str(metrics.f1_score(dev_labels, dev_pred_labels_knn, average="weighted")))
  
  search_mnb = MultinomialNB()
  #in broad search 0.01 was best param, this is a finer grained search to see if we can do better.
  params_mnb = {'alpha': [n / 1000.0 for n in range(1, 100, 1)]}
  gscv_mnb = GridSearchCV(search_mnb, params_mnb, cv=5, scoring='f1_weighted')
  gscv_mnb.fit(train_vectors, train_labels)
  print("Best alpha-Value for MultinomialNB via GridSearchCV: " + str(gscv_mnb.best_params_))
  dev_pred_labels_mnb = gscv_mnb.predict(dev_vectors)
  print("f1-score for MultinomialNB: " + str(metrics.f1_score(dev_labels, dev_pred_labels_mnb, average="weighted")))
  
  search_lr = LogisticRegression(solver='liblinear', multi_class='auto')
  params_lr = {'C': [n / 100.0 for n in range(40, 60)]}
  gscv_lr = GridSearchCV(search_lr, params_lr, cv=5, scoring='f1_weighted')
  gscv_lr.fit(train_vectors, train_labels)
  print("Best C-Value for LogisticRegression via GridSearchCV: " + str(gscv_lr.best_params_))
  dev_pred_labels_lr = gscv_lr.predict(dev_vectors)
  print("f1-score for LogisticRegression: " + str(metrics.f1_score(dev_labels, dev_pred_labels_lr, average="weighted")))
  
  for c in [n / 100.0 for n in range(40,60,5)]:
    lr = LogisticRegression(C=c, solver='liblinear', multi_class='auto')
    lr.fit(train_vectors, train_labels)
    print("Sum of squared weights for C = " + str(c) + ":")
    for i in range(len(lr.coef_)):
      vect = lr.coef_[i]
      class_name = newsgroups_train.target_names[i]
      sum_sq = sum(n*n for n in vect)
      print("For " + class_name + ": " + str(sum_sq))
      

P3()

ANSWER:


1.   Why doesn't nearest neighbors work well for this problem?

   a.   The main reason nearest neighbors does not work well is because of the large volume and variation of common words used throughout the documents. This causes an given document to match other documents that share a similar corpus of common words rather than taking into account the less common words which likely have more relevance to the meaning of the document. This is especially true due to the high frequency of common words in a given document compared to 1 or 2 instances of words that may better align with other documents of the same category.

2.   Any ideas why logistic regression doesn't work as well as Naive Bayes?

   b.   There are a couple of reasons why I think Naive Bayes works better for this problem than Logistic Regression. Firstly, Logistic Regression tends to not perform as well with sparse data because it becomes hard to converge on the best parameter values. Related to this, Naive Bayes on the other hand treats each feature independently allowing words that only appear in a given class in the training data to have a much higher impact than in Logistic Regression where you are trying to fit all coefficients together as a single model.
   
3.   As we can see above the sum of squared coefficients for a given class increase as we increase the C parameter. This is because C is the inverse of the regularization strength, $\lambda$. This means that as C increases, the regularization strength decreases which reduces the penalty of having large coefficients and therefore allows the sum of squared weights to be higher.

### Part 4:

Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [0]:
def get_feature_table(vectorizer, linear_model, feats_per_class):
  train_vectors = vectorizer.fit_transform(train_data)
  train_features = vectorizer.get_feature_names()
  linear_model.fit(train_vectors, train_labels)
  
  best_features = []
  for i in range(len(linear_model.coef_)):
    vect = linear_model.coef_[i]
    class_name = newsgroups_train.target_names[i]
    best_features_for_class = heapq.nlargest(feats_per_class, enumerate(vect), key=lambda x: x[1])
    print("Best features for " + class_name + ": " + str([train_features[feat[0]] for feat in best_features_for_class]))
    best_features.extend(feat[0] for feat in best_features_for_class)
    
  feature_table_data = []
  for feat in best_features:
    feat_dict = {'feature_gram': train_features[feat]}
    for i in range(len(linear_model.coef_)):
      vect = linear_model.coef_[i]
      class_name = newsgroups_train.target_names[i]
      weight = vect[feat]
      feat_dict[class_name + "_weight"] = weight
    feature_table_data.append(feat_dict)
    
  return pd.DataFrame(feature_table_data, columns=[class_name + "_weight" for class_name in newsgroups_train.target_names], index=[feat['feature_gram'] for feat in feature_table_data])
  

def P4():
  feats_per_class = 5
  lr_c = 0.53
  
  lr = LogisticRegression(C=lr_c, solver='liblinear', multi_class='auto', max_iter = 1000)# failed to converge on bi-grams with max_iter=100
  
  vectorizer = CountVectorizer()
  feature_table = get_feature_table(vectorizer, lr, feats_per_class)
  display(HTML(feature_table.to_html()))
  
  bi_vectorizer = CountVectorizer(analyzer = 'char', ngram_range = (2,2))
  bi_feature_table = get_feature_table(bi_vectorizer, lr, feats_per_class)
  display(HTML(bi_feature_table.to_html()))

P4()

ANSWER: 

For the ngram word features the most surprising results for me are as follows:

1.   fbi (talk.religion.misc) - Seems strange that talk of the FBI would be an indicator of miscellaneous religious messages.

2.   deletion, religion, bobby (alt.atheism) - While religion does make some sense with atheism, it is somewhat surprising that the difference in coefficients between alt.atheism and talk.religion.misc is so stark. Deletion and bobby seem like unrelated additions and bobby in particular makes me worried there is some overfitting going on (possibly due to the name of a common author on the subject in the training corpus).

For the bi-gram features the comp.graphics features seem to be the only ones that make logical sense due to the inclusion of 3d and some common file name extensions (cs, py). It is surprising that punctation seems to be a strong indicator in a number of cases (i.e. '; ' and 'n:' for religion, '\nn' for space, and 's:' for atheism).

### Part 5:

Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [0]:
def empty_preprocessor(s):
  return s

def better_preprocessor(s):
  s = unescape(s)
  if(not s.isupper()):
    s = s.lower()
  #elif(len(s) > 1):
  #  s = s.lower() + ' THISISALLCAPS'
  #s = re.sub('(relig)\w+', r'\1', s)
  s = re.sub('(athe)\w+', r'\1', s)#s = re.sub('^.+(athe).+$', r'\1', s)
  s = re.sub('(\w+)y', r'\1', s)
  s = re.sub('(\w+)ies', r'\1', s)
  s = re.sub('(\w+)es', r'\1', s)
  s = re.sub('(\w+)s', r'\1', s)
  s = re.sub('(\w+)ed', r'\1', s)
  #s = re.sub('-', r' ', s)
  #s = re.sub('^(\w+)\'d$', r'\1', s)
  #s = re.sub('^(\w+)\'s$', r'\1', s)
  #s = re.sub('^(\w+)\'ll$', r'\1', s)
  s = re.sub('\d+', '_A_NUMBER_', s)
  #s = re.sub('^\d{2}$', '##', s)
  #s = re.sub('^\d{3}$', '###', s)
  #s = re.sub('^\d+$', '####', s)
  return s

def get_f1_and_vocab(preprocessor, linear_model):
  train_vectorizer = CountVectorizer(preprocessor=preprocessor)
  train_vectors = train_vectorizer.fit_transform(train_data)
  train_features = train_vectorizer.get_feature_names()
  dev_vectorizer = CountVectorizer(preprocessor=preprocessor, vocabulary=train_features)
  dev_vectors = dev_vectorizer.fit_transform(dev_data)
  linear_model.fit(train_vectors, train_labels)
  dev_pred_labels = linear_model.predict(dev_vectors)
  return metrics.f1_score(dev_labels, dev_pred_labels, average="weighted"), train_features

def P5():
  lr_c = 0.53
  lr = LogisticRegression(C=lr_c, solver='liblinear', multi_class='auto')
  
  f1_empty, vocab_empty = get_f1_and_vocab(empty_preprocessor, lr)
  print("f1-score with empty preprocessor: " + str(f1_empty))
  print("Vocab size with empty preprocessor: " + str(len(vocab_empty)))
  
  f1_better, vocab_better = get_f1_and_vocab(better_preprocessor, lr)
  print("f1-score with better preprocessor: " + str(f1_better))
  print("Vocab size with better preprocessor: " + str(len(vocab_better)))
  
  print("Improvement: " + str(f1_better - f1_empty))
  print("Vocab size reduction: " + str(len(vocab_empty) - len(vocab_better)))
  
P5()

### Part 6:

The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.015 (the default is .0001).

In [0]:
def cnt_nonzero_weights(linear_model):
  cnt = 0
  for vect in linear_model.coef_:
    for n in vect:
      if n != 0.0:
        cnt += 1
  return cnt

def generate_filtered_model(train_vectorizer, c):
  train_vectors = train_vectorizer.fit_transform(train_data)
  train_features = train_vectorizer.get_feature_names()
  lr_l1 = LogisticRegression(C=c, penalty='l1', solver='liblinear', multi_class='auto', tol=0.015)
  lr_l1.fit(train_vectors, train_labels)
  feature_index_set = set()
  for vect in lr_l1.coef_:
    for i in range(len(vect)):
      if vect[i] != 0:
        feature_index_set.add(i)
        
  new_vocab = [train_features[i] for i in feature_index_set]
  new_vectorizer = CountVectorizer(vocabulary=new_vocab)
  new_vectors = new_vectorizer.fit_transform(train_data)
  new_features = new_vectorizer.get_feature_names()
  lr_l2 = LogisticRegression(C=0.53, solver='liblinear', multi_class='auto')
  lr_l2.fit(new_vectors, train_labels)
  return new_vectorizer, lr_l2
  

def P6():
  np.random.seed(0)
  
  train_vectorizer = CountVectorizer()
  train_vectors = train_vectorizer.fit_transform(train_data)
  train_features = train_vectorizer.get_feature_names()
  
  lr_c = 0.53
  lr_l2 = LogisticRegression(C=lr_c, solver='liblinear', multi_class='auto')
  lr_l2.fit(train_vectors, train_labels)
  cnt_nz_l2 = cnt_nonzero_weights(lr_l2)
  print("Number non-zero coefficients with L2: " + str(cnt_nz_l2))
  
  lr_l1 = LogisticRegression(C=lr_c, penalty='l1', solver='liblinear', multi_class='auto', tol=0.015)
  lr_l1.fit(train_vectors, train_labels)
  cnt_nz_l1 = cnt_nonzero_weights(lr_l1)
  print("Number non-zero coefficients with L1: " + str(cnt_nz_l1))
  
  x_vocabsize = []
  y_acc = []
  for c in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
    vectorizer, model = generate_filtered_model(train_vectorizer, c)
    dev_vectors = vectorizer.fit_transform(dev_data)
    dev_pred_labels = model.predict(dev_vectors)
    x_vocabsize.append(len(model.coef_[0]))
    y_acc.append(metrics.accuracy_score(dev_labels, dev_pred_labels))
    
  #print(x_vocabsize)
  #print(y_acc)
  fig_size = 10
  fig = plt.figure(figsize=(fig_size, fig_size))
  ax1 = fig.add_subplot(1, 1, 1)
  ax1.scatter(x_vocabsize, y_acc)
  
  ax1.set_xlabel('vocab_size')
  ax1.set_ylabel('accuracy_score')
  ax1.set_title('vocab size vs accuracy score (for increasing C-values)')
  
  plt.show()
    
    
P6()

### Part 7:

Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [0]:
def P7():
  c = 100
  n_docs = 3
  
  train_vectorizer = TfidfVectorizer()
  train_vectors = train_vectorizer.fit_transform(train_data)
  train_features = train_vectorizer.get_feature_names()
  
  lr = LogisticRegression(C=c, solver='liblinear', multi_class='auto')
  lr.fit(train_vectors, train_labels)
  dev_vectorizer = TfidfVectorizer(vocabulary=train_features)
  dev_vectors = dev_vectorizer.fit_transform(dev_data)
  dev_pred_proba = lr.predict_proba(dev_vectors)
  
  large_R = heapq.nlargest(n_docs, enumerate(dev_pred_proba)
                 , key=lambda x: max(x[1])/(x[1][dev_labels[x[0]]]))
  
  for doc in large_R:
    print("------------------------------------------------------------------")
    print("Predicted: " + newsgroups_train.target_names[np.argmax(doc[1])]
          + ", Actual: " + newsgroups_train.target_names[dev_labels[doc[0]]])
    print("------------------------------------------------------------------")
    print(dev_data[doc[0]] + "\n\n")
    
  
  print("\n\n Error Analysis:\n")
  ftp_ix = train_features.index('ftp')
  graph_ix = newsgroups_train.target_names.index('comp.graphics')
  ftp_graph_coef = lr.coef_[graph_ix][ftp_ix]
  print("coefficient value for ftp in comp.graphics: " + str(ftp_graph_coef))
  print("This coefficient is in the top {:.2f}% of coefficients for comp.graphics.".format(percentileofscore(lr.coef_[graph_ix], ftp_graph_coef)))
  
  print("\n\n")
  print("Word and coefficient for words with the highest coefficients for the predicted class of top R-value documents")
  for doc in large_R:
    vector = dev_vectors[doc[0]].toarray()[0]
    words = []
    for word in enumerate(vector):
      if word[1] > 0:
        words.append(word[0])
    
    word_coefs = []
    for word in words:
      pred_coef = lr.coef_[np.argmax(doc[1])]
      word_coefs.append((train_features[word], pred_coef[word]))
      
    n_strongest = 10
    print(heapq.nlargest(n_strongest, word_coefs, key=lambda x: x[1]))
  
P7()

ANSWER: The output above shows which words may be contributing the most to a given miscategorization. The first and third document shown have a few words in common that may be why they are both miscategorized as comp.graphics. Namely, 'version' and 'ftp'. In order to fix this we could play around with stop_words in an attempt to manually filter out words that are causing issues.

### Part 8 EXTRA CREDIT:

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

In [0]:
def P8():
  c = 100
  n_docs = 3
  
  train_vectorizer = TfidfVectorizer(stop_words=['ftp', 'version'])
  train_vectors = train_vectorizer.fit_transform(train_data)
  train_features = train_vectorizer.get_feature_names()
  
  lr = LogisticRegression(C=c, solver='liblinear', multi_class='auto')
  lr.fit(train_vectors, train_labels)
  dev_vectorizer = TfidfVectorizer(vocabulary=train_features)
  dev_vectors = dev_vectorizer.fit_transform(dev_data)
  dev_pred_proba = lr.predict_proba(dev_vectors)
  
  large_R = heapq.nlargest(n_docs, enumerate(dev_pred_proba)
                 , key=lambda x: max(x[1])/(x[1][dev_labels[x[0]]]))
  
  for doc in large_R:
    print("------------------------------------------------------------------")
    print("Predicted: " + newsgroups_train.target_names[np.argmax(doc[1])]
          + ", Actual: " + newsgroups_train.target_names[dev_labels[doc[0]]])
    print("------------------------------------------------------------------")
    print(dev_data[doc[0]] + "\n\n")
    
  print("\n\n")
  
  for doc in large_R:
    vector = dev_vectors[doc[0]].toarray()[0]
    words = []
    for word in enumerate(vector):
      if word[1] > 0:
        words.append(word[0])
    
    word_coefs = []
    for word in words:
      pred_coef = lr.coef_[np.argmax(doc[1])]
      word_coefs.append((train_features[word], pred_coef[word]))
      
    n_strongest = 10
    print(heapq.nlargest(n_strongest, word_coefs, key=lambda x: x[1]))
  
P8()